In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/mnist/", one_hot=True)

Extracting /tmp/mnist/train-images-idx3-ubyte.gz
Extracting /tmp/mnist/train-labels-idx1-ubyte.gz
Extracting /tmp/mnist/t10k-images-idx3-ubyte.gz
Extracting /tmp/mnist/t10k-labels-idx1-ubyte.gz


In [2]:
import tensorflow as tf

print(tf.convert_to_tensor(mnist.train.images).get_shape())
print(tf.convert_to_tensor(mnist.test.images).get_shape())
print(tf.convert_to_tensor(mnist.train.images).get_shape().as_list()[0])

(55000, 784)
(10000, 784)
55000


In [3]:
# Set parameters
learning_rate = 0.01
training_iteration = 30
batch_size = 100
display_step = 2
summary = True

# Additional variables
train_size = tf.convert_to_tensor(mnist.train.images).get_shape().as_list()[0]

# TF graph input
x = tf.placeholder("float", [None, 784])
y = tf.placeholder("float", [None, 10])

# Create a model
# Set model weights
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))


with tf.name_scope("Wx_b") as scope:
    # Construct a linear model
    model = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax

# Add summary ops to collect data
w_h = tf.summary.histogram("weights", W)
b_h = tf.summary.histogram("biases", b)

# More name scopes 
with tf.name_scope("cost_fuction") as scope:
    # Minimize error using cross entropy
    cost_function = -tf.reduce_sum(y * tf.log(model))
    # Create a summmary to monitor the cost function
    tf.summary.scalar("cost_function", cost_function)
    
with tf.name_scope("train") as scope:
    # Gradient descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)

In [4]:
# Initializing the variables
init = tf.global_variables_initializer()

# Merge all summaries into a single operator
merged_summary_op = tf.summary.merge_all()


# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    # Change this to a location on your computer
    if summary:
        summary_writer = tf.summary.FileWriter('/tmp/graph', graph=sess.graph)
    
    # training cycle
    for iteration in range(training_iteration):
        avg_cost = 0.
        total_batch = int(train_size / batch_size)
        # Loop over all batches
        for i in range(total_batch):
            # 훈련 데이터셋으로 부터 무작위로 100개를 추출
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Fit training using batch data
            sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys})
            # Compute the average loss
            avg_cost += sess.run(cost_function, feed_dict={x: batch_xs, y: batch_ys})/total_batch
            # Write logs for each iteration
            summary_str = sess.run(merged_summary_op, feed_dict={x: batch_xs, y: batch_ys})
            if summary:
                summary_writer.add_summary(summary_str, iteration * total_batch + 1)
        # Display logs per iteration step
        if iteration % display_step == 0:
            print("Iteration:", '%04d' % (iteration + 1), "cost=", "{:.9f}".format(avg_cost))
    
    print("Tuning completed!")
    
    # Test the model
    predictions = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(predictions, "float"))

    train_accuracy = accuracy.eval({x: mnist.train.images, y: mnist.train.labels})
    test_accuracy = accuracy.eval({x: mnist.test.images, y: mnist.test.labels})

    print(train_accuracy, test_accuracy)            

Iteration: 0001 cost= 29.860465115
Iteration: 0003 cost= 21.088599887
Iteration: 0005 cost= 20.125664861
Iteration: 0007 cost= 19.607141489
Iteration: 0009 cost= 19.333187017
Iteration: 0011 cost= 19.029060418
Iteration: 0013 cost= 18.981711084
Iteration: 0015 cost= 18.700364337
Iteration: 0017 cost= 18.494846365
Iteration: 0019 cost= 18.630231689
Iteration: 0021 cost= 18.389173530
Iteration: 0023 cost= 18.296941533
Iteration: 0025 cost= 18.331790807
Iteration: 0027 cost= 18.116435575
Iteration: 0029 cost= 18.068089578
Tuning completed!
0.928255 0.9197
